In [8]:
import gymnasium as gym
import numpy as np

In [9]:
env = gym.make('FrozenLake-v1', is_slippery = False, render_mode = 'human')
env = env.unwrapped
env.reset()

# Parameters
gamma = 0.8
theta = 1e-8

In [10]:
# Supporting functions
def policy_evaluation(env, policy, theta=1e-8, gamma=0.9):
    V = np.zeros(env.observation_space.n)
    while True:
        delta = 0

        for s in range(env.observation_space.n):
            
            old_v = V[s]
           
            new_v = 0
            for a, action_prob in enumerate(policy[s]):
                if action_prob > 0:
                    transitions = env.P[s][a]
                    for p, ns, r, _ in transitions:
                        new_v += action_prob * p * (r + gamma * V[ns])
            
            
            V[s] = new_v
            delta = max(delta, np.abs(old_v - V[s]))
        if delta < theta:
            break
    return V

def policy_improvement(env, V, gamma=0.9):
    policy = np.zeros([env.observation_space.n, env.action_space.n])
    for s in range(env.observation_space.n):
        q_values = np.zeros(env.action_space.n)
        for a in range(env.action_space.n):
            q_values[a] = sum([p * (r + gamma * V[ns]) for p, ns, r, _ in env.P[s][a]])
        best_action = np.argmax(q_values)
        policy[s, best_action] = 1.0
    return policy

def policy_iteration(env, theta=1e-8, gamma=0.9):
    policy = np.ones([env.observation_space.n, env.action_space.n]) / env.action_space.n
    while True:
        V = policy_evaluation(env, policy, theta, gamma)
        new_policy = policy_improvement(env, V, gamma)
        if np.array_equal(policy, new_policy):
            break
        policy = new_policy
    return policy, V

def value_iteration(env, theta=1e-8, gamma=0.9):
    V = np.zeros(env.observation_space.n)
    while True:
        delta = 0
        for s in range(env.observation_space.n):
            q_values = [sum([p * (r + gamma * V[ns]) for p, ns, r, _ in env.P[s][a]]) for a in range(env.action_space.n)]
            max_q_value = max(q_values)
            delta = max(delta, abs(V[s] - max_q_value))
            V[s] = max_q_value
        if delta < theta:
            break
    policy = policy_improvement(env, V, gamma)
    return policy, V

# Run policy iteration
pi_policy, pi_value = policy_iteration(env, theta=1e-8, gamma=0.9)
vi_policy, vi_value = value_iteration(env, theta=1e-8, gamma=0.9)
print("\nPolicy from Policy Iteration:")

# Simulate using the optimal policy
def simulate_optimal_policy(env, policy, max_steps=100):
    """
    Simulate an episode in the Taxi environment using the optimal policy.
    
    Args:
        env: The Taxi environment.
        policy: The optimal policy from policy or value iteration.
        max_steps: Maximum steps to simulate.

    Returns:
        total_reward: Total accumulated reward during simulation.
        path: List of states visited.
        actions: List of actions taken.
    """
    state = env.reset()[0]  # Reset environment and get initial state
    total_reward = 0
    path = [state]  # Track states visited
    actions = []  # Track actions taken

    for _ in range(max_steps):
        action = np.argmax(policy[state])  # Best action from the policy
        next_state, reward, done, _, _ = env.step(action)  # Step in environment

        total_reward += reward
        path.append(next_state)
        actions.append(action)

        state = next_state
        env.render()  # Optional: Render each step

        if done:
            break

    return total_reward, path, actions


Policy from Policy Iteration:


In [11]:
# Using the optimal policy to simulate (Policy Iteration)
total_reward, path, actions = simulate_optimal_policy(env, pi_policy)
print("Total Reward:", total_reward)
print("Path taken:", path)
print("Actions taken:", actions)
#env.close()

Total Reward: 1.0
Path taken: [0, 4, 8, 9, 13, 14, 15]
Actions taken: [np.int64(1), np.int64(1), np.int64(2), np.int64(1), np.int64(2), np.int64(2)]


In [12]:
# Using the optimal policy to simulate (Value Iteration)
total_reward, path, actions = simulate_optimal_policy(env, vi_policy)
print("Total Reward:", total_reward)
print("Path taken:", path)
print("Actions taken:", actions)
env.close()

Total Reward: 1.0
Path taken: [0, 4, 8, 9, 13, 14, 15]
Actions taken: [np.int64(1), np.int64(1), np.int64(2), np.int64(1), np.int64(2), np.int64(2)]


In [13]:
def print_policy(policy, env):
    env = env.unwrapped
    desc = env.desc.astype(str)
    nrow, ncol = desc.shape
    policy_actions = np.argmax(policy, axis=1)

    symbols = {0: '←', 1: '↓', 2: '→', 3: '↑'}

    for i in range(nrow):
        for j in range(ncol):
            s = i * ncol + j
            if desc[i, j] == 'G':
                print('G', end=' ')
            elif desc[i, j] == 'H':
                print('X', end=' ')
            else:
                action = policy_actions[s]
                print(symbols[action], end=' ')
        print()

def print_value_function(V, env):
    env = env.unwrapped
    desc = env.desc.astype(str)
    nrow, ncol = desc.shape

    for i in range(nrow):
        for j in range(ncol):
            s = i * ncol + j
            if desc[i, j] == 'H':
                print('  X  ', end=' ')
            else:
                print(f'{V[s]:5.2f}', end=' ')
        print()

In [14]:
# Example usage:
env = gym.make('FrozenLake-v1', render_mode='human')
observation, info = env.reset()

print("Policy Iteration:")


print("Optimal Policy:")
print_policy(pi_policy, env)
print("\nOptimal Value Function:")

print_value_function(pi_value, env)

print("\nValue Iteration:")


print("Optimal Policy:")
print_policy(vi_policy, env)
print("\nOptimal Value Function:")
print_value_function(vi_value, env)

env.close()

Policy Iteration:
Optimal Policy:
↓ → ↓ ← 
↓ X ↓ X 
→ ↓ ↓ X 
X → → G 

Optimal Value Function:
 0.59  0.66  0.73  0.66 
 0.66   X    0.81   X   
 0.73  0.81  0.90   X   
  X    0.90  1.00  0.00 

Value Iteration:
Optimal Policy:
↓ → ↓ ← 
↓ X ↓ X 
→ ↓ ↓ X 
X → → G 

Optimal Value Function:
 0.59  0.66  0.73  0.66 
 0.66   X    0.81   X   
 0.73  0.81  0.90   X   
  X    0.90  1.00  0.00 
